In [ ]:
%run ../ImportsConstantsSparkInit.ipynb
sc.setJobGroup("DA_Covid", "DA_Covid")

In [ ]:
%%time
#First year of Covid (March 11 2020) +/- 2 months on either end
pipeline = "[{ $match :{Start_Time: {$gte:ISODate('2020-01-11'), $lt:ISODate('2021-05-12')}}}, { $group : { _id : {$dateToString: { format: '%Y-%m', date: '$Start_Time' }}, count:{$sum:1}}}, {$project:{count:1, Month:{$substr:['$_id', 5,7]}, Year:{$substr:['$_id', 0,4]}}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
df_temp = df.groupby("_id").sum("count")
df = df.join(df_temp, df._id == df_temp._id)
df = df.drop("count")
df = df.dropDuplicates()
df = df.withColumnRenamed("_id","Date").withColumnRenamed("sum(count)","AccidentCount")
df = df.drop("Date")
print("Number of Documents: " + str(df.count()))
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)

In [ ]:
df.write.format("mongo").option("collection", "FirstYearOfCovidAccidents").mode("overwrite").save()